In [3]:
!pip install deep-sort-realtime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 78.9 MB/s eta 0:00:00


In [4]:
import cv2
import numpy as np
from deep_sort_realtime.deepsort_tracker import DeepSort
from google.colab import files

# Load SSD MobileNet
net = cv2.dnn.readNetFromCaffe(
    '/content/MobileNetSSD_deploy.prototxt',
    '/content/MobileNetSSD_deploy.caffemodel'
)

# Class labels
CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat",
           "bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
           "dog", "horse", "motorbike", "person", "pottedplant", "sheep",
           "sofa", "train", "tvmonitor"]
VEHICLE_CLASSES = {"car"}

# Initialize Deep SORT
tracker = DeepSort(max_age=10, n_init=3, nms_max_overlap=1.0)

# Load video
cap = cv2.VideoCapture('videoplayback.mp4')
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

out = cv2.VideoWriter('output_deepsort.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

id_mapping = {}
next_compact_id = 0
total_vehicles = set()

CONFIDENCE_THRESHOLD = 0.20  # lowered to catch more vehicles
MIN_BOX_AREA = 300  # slightly reduced for smaller/far vehicles

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    h, w = frame.shape[:2]

    # Optional upscale for better detection
    upscale_factor = 1.2
    resized_frame = cv2.resize(frame, None, fx=upscale_factor, fy=upscale_factor)
    rh, rw = resized_frame.shape[:2]

    blob = cv2.dnn.blobFromImage(resized_frame, 0.007843, (300, 300), 127.5)
    net.setInput(blob)
    detections = net.forward()

    dets_for_sort = []

    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > CONFIDENCE_THRESHOLD:
            idx = int(detections[0, 0, i, 1])
            class_name = CLASSES[idx]
            if class_name in VEHICLE_CLASSES:
                box = detections[0, 0, i, 3:7] * np.array([rw, rh, rw, rh])
                x1, y1, x2, y2 = box.astype(int)

                # Downscale box back to original frame size
                x1 = int(x1 / upscale_factor)
                y1 = int(y1 / upscale_factor)
                x2 = int(x2 / upscale_factor)
                y2 = int(y2 / upscale_factor)

                if (x2 - x1) * (y2 - y1) < MIN_BOX_AREA:
                    continue

                dets_for_sort.append(([x1, y1, x2 - x1, y2 - y1], confidence, class_name))

    # Optional: You could do cv2.dnn.NMSBoxes here if needed

    tracks = tracker.update_tracks(dets_for_sort, frame=frame)

    for track in tracks:
        if not track.is_confirmed():
            continue

        track_id = track.track_id
        if track_id not in id_mapping:
            id_mapping[track_id] = next_compact_id
            next_compact_id += 1

        compact_id = id_mapping[track_id]
        total_vehicles.add(compact_id)

        x1, y1, x2, y2 = map(int, track.to_ltrb())

        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"ID: {compact_id}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)

    cv2.putText(frame, f"Total Vehicles: {len(total_vehicles)}", (20, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    out.write(frame)

cap.release()
out.release()
files.download('output_deepsort.mp4')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>